In [ ]:
import matplotlib.pyplot as plt
import datetime
import tilemapbase as tmb
import numpy as np
import pandas as pd
from pluma.stream.georeference import Georeference
from pluma.stream.ubx import _UBX_MSGIDS
from pluma.schema import Dataset

## Notebook plotting
%matplotlib inline
plt.style.use('ggplot')

## Figure export parameters
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
import matplotlib as mpl
mpl.rcParams.update(new_rc_params)

In [ ]:
stream_root_folder = r's3://emotional-cities/data/nrg/poc-v1/TechnicalBenchmarks/EmpaticaLDRBenchmark2022_09_26_18_06_45'
from benchmark_schemas import build_benchmarkschema_empatica

dataset = Dataset(
    stream_root_folder,
    datasetlabel="Empatica_Benchmark",
    georeference= Georeference(),
    schema=build_benchmarkschema_empatica)
dataset.populate_streams(autoload=True)



In [ ]:
rolling_max = dataset.streams.BioData.ECG.data["Value1"].rolling('1s').max() #get rid of double hits due to hysteresis using a max-window filter.


thr = 70
ttl = (rolling_max > thr).astype(int)
pd_rising_edge = ttl[np.where(np.diff(ttl, append=np.nan) == 1)[0]]

fig = plt.figure(figsize= (15,4))
plt.plot(dataset.streams.BioData.ECG.data["Value1"]/1000, label = 'RawSignal')
plt.plot(ttl, label = 'ThresholdedSignal')
plt.scatter(x = pd_rising_edge.index, y = pd_rising_edge.values*0 + 0.5, color = "darkgreen", label = 'risingEdge')
plt.plot(rolling_max/1000, label = 'Filtered signal')
plt.ylabel('Voltage (adc units)')
plt.xlabel('Time')
plt.scatter(dataset.streams.Empatica.data.E4_Tag.index,  np.ones((len(dataset.streams.Empatica.data.E4_Tag.index),1)) * 0.6, label = "EmpaticaTags")

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 0.95),
          ncol=3, fancybox=True, shadow=True)


plt.xlim((pd_rising_edge.index[82], pd_rising_edge.index[88]))
plt.show()

#Confirm no double hits were detected. There should be no events below 1 second.
plt.figure(figsize= (3,2))
plt.hist(np.diff(pd_rising_edge.index.values)/ np.timedelta64(1, 's'), np.linspace(0, 10, 20))
plt.xlabel('$\Delta t_{OnSet}$')
plt.ylabel('Counts')
plt.title('Refractory period violations')
plt.show()


In [ ]:
delta_t = pd_rising_edge.index.values - dataset.streams.Empatica.data.E4_Tag.index.values
delta_t = -delta_t / np.timedelta64(1, 'ms')
delta_t = delta_t[delta_t > 0]

plt.figure(figsize= (9,6))
plt.hist(delta_t, bins = 25, color= "Teal")
plt.xlabel('Latency (ms)')
plt.ylabel('Counts')
plt.xlim((0,250))
plt.title(f'Empatica latency benchmark\n$\mu = {np.mean(delta_t):.2f}, \sigma = {np.std(delta_t):.2f},$ [min:max] = [{np.min(delta_t):.2f}:{np.max(delta_t):.2f}]')
plt.show()